# Setting

In [4]:
# function ClickConnect(){
# console.log("Working"); 
# document.querySelector("colab-toolbar-button#connect").click() 
# }setInterval(ClickConnect, 1800000)

In [5]:
from google.colab import drive
drive.mount('/content/drive')

import os
REF_PATH = '/content/drive/MyDrive/Github/10_도배하자유형분류'
os.chdir(REF_PATH)

Mounted at /content/drive


<br>

## Import

In [6]:
import gc
gc.collect()

0

In [7]:
import os,sys
import random
import joblib
import pandas as pd
import numpy as np
import glob
import cv2
import itertools

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
# import torchvision.models as models

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from tqdm import tqdm, trange

import warnings
warnings.filterwarnings(action='ignore') 

from PIL import Image

<br>

## Hyperparameter Setting

In [8]:
MC_PATH = './mc/stylegan'
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda


In [9]:
CFG = {
    'IMG_SIZE':224, #224,320,384
    'EPOCHS':128,
    'LEARNING_RATE':3e-4,
    'BATCH_SIZE':16,
    'SEED':41,
}

<br>

## Fixed RandomSeed

In [10]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

<br></br>

# Data Load

<br>

## Data Pre-processing

In [11]:
all_img_list = glob.glob('./data/train/*/*')

In [12]:
df = pd.DataFrame(columns=['img_path', 'label'])
df['img_path'] = all_img_list
df['label'] = df['img_path'].apply(lambda x : x.split('/')[-2].replace('.png',''))

In [13]:
# train_data, val_data, _, _ = train_test_split(df, df['label'], test_size=0.3, stratify=df['label'], random_state=CFG['SEED'])

<br>

## Label-Encoding

In [14]:
le = preprocessing.LabelEncoder()
# train_data['label'] = le.fit_transform(train_data['label'])
# val_data  ['label'] = le.transform(val_data['label'])
df['label'] = le.fit_transform(df['label'])

In [15]:
for i,label in enumerate(le.classes_):
    print(i,label)

0 가구수정
1 걸레받이수정
2 곰팡이
3 꼬임
4 녹오염
5 들뜸
6 면불량
7 몰딩수정
8 반점
9 석고수정
10 오염
11 오타공
12 울음
13 이음부불량
14 창틀,문틀수정
15 터짐
16 틈새과다
17 피스
18 훼손


In [16]:
# 한글을 le.inverse_transform() 할 경우에 안맞는 현상생김
# -> 손코딩 필요
def label_decoder(label):
    new_label =[
        '가구수정' if l==0 else
        '걸레받이수정' if l==1 else
        '곰팡이' if l==2 else
        '꼬임' if l==3 else
        '녹오염' if l==4 else
        '들뜸' if l==5 else
        '면불량' if l==6 else
        '몰딩수정' if l==7 else
        '반점' if l==8 else
        '석고수정' if l==9 else
        '오염' if l==10 else
        '오타공' if l==11 else
        '울음' if l==12 else
        '이음부불량' if l==13 else
        '창틀,문틀수정' if l==14 else
        '터짐' if l==15 else
        '틈새과다' if l==16 else
        '피스' if l==17 else
        '훼손' if l==18 else
        'NaN' for l in label
    ]
    return np.array(new_label)

<br></br>

# Image Generate by GAN

## Pre-trained Model

In [17]:
def mkdir(path):
    import os
    if not os.path.exists(path):
        print("folder created: {}".format(path))
        os.mkdir(path)

In [18]:
# # 기존파일 삭제
# import shutil
# shutil.rmtree(f'./out/gan_images')
# shutil.rmtree(f'./mc/gan')

In [19]:
# 학습 이미지 저장을 위한 디렉토리 생성
mkdir(f'./out')
mkdir(f'./out/gan_images')
mkdir(f'./out/gan_images/train')
mkdir(f'./out/gan_images/generate')

# 결과파일 저장을 위한 디렉토리 생성
mkdir(f'./mc/stylegan')

folder created: ./mc/stylegan


In [52]:
from tqdm.auto import tqdm
import torch
import torchvision.transforms as transforms
from torch.utils.data import Dataset
from PIL import Image

class ImageFolder(Dataset):
    def __init__(self, img_path_list):
        self.transform = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(256),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
        ])
        
        self.images = []
        for img_path in tqdm(img_path_list):
            image = Image.open(img_path)
            image = self.transform(image)
            self.images.append(image)

    def __getitem__(self, index):
        return self.images[index]

    def __len__(self):
        return len(self.images)

In [53]:
import torch
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from stylegan2_pytorch import ModelLoader, Trainer

# 내가 가진 이미지를 포함한 데이터셋을 준비합니다.
model_loader = ModelLoader(base_dir='./data/train/가구수정/')
# model_loader.load_state_dict('path/to/pretrained_model.pth')
ImageDataset = ImageFolder(img_path_list=glob.glob('./data/train/가구수정/*.png'))
dataloader = DataLoader(ImageDataset,batch_size=8,shuffle=True,num_workers=2,pin_memory=True)

  0%|          | 0/12 [00:00<?, ?it/s]

In [58]:
pip install stylegan2-ada-pytorch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement stylegan2-ada-pytorch (from versions: none)
ERROR: No matching distribution found for stylegan2-ada-pytorch


In [56]:
import torch
import os
from pathlib import Path
from typing import List
from PIL import Image
from torchvision.transforms.functional import to_tensor
from stylegan2_ada_pytorch import legacy

ModuleNotFoundError: ignored

In [ ]:
def main_iteration(label,epochs,metric_peroid,save_period):
    gc.collect()

    generator_save_path     = MC_PATH + f'/generator_{label}.pt'
    image_save_path         = f'./out/gan_images/train/{label}/'
    gen_image_save_path     = f'./out/gan_images/generate/{label}/'
    mkdir(image_save_path)
    mkdir(gen_image_save_path)

    sub_data = train_data[train_data.label==label]
    # Image.open(sub_data.img_path.values[0])
    train_dataset = GANDataset(img_path_list=sub_data.img_path.values)
    train_loader  = DataLoader(train_dataset, batch_size=32, num_workers=0, shuffle=True)

    # GAN의 목적은 Discriminator loss를 높이는 것과 Generator loss를 낮추는 것
    image_gen = ImageGenerator()

    # GAN 모델 학습
    image_gen.train(
        epochs=epochs,
        dataloader=train_loader,
        lr=5e-4,
        device=device,
        metric_period=metric_peroid,
        save_period=save_period,
        generator_save_path=generator_save_path,
        image_save_path=image_save_path,
    )

    # GAN 모델을 통해서 이미지생성
    image_gen.generate(
        n_generate=100,
        save_path=gen_image_save_path,
    )

    return image_gen

In [ ]:
vc = train_data.label.value_counts()
label_list = vc[vc<vc.max()].index.tolist()
label_list = sorted(label_list)

# for iter,label in enumerate(label_list):
for iter,label in enumerate([17]):
    print()
    print('-'*80)
    print('> ({}/{}) {}'.format(iter+1,len(label_list),label))
    print('-'*80)
    main_iteration(label=label,epochs=5000,metric_peroid=500,save_period=1000)


--------------------------------------------------------------------------------
> (1/18) 17
--------------------------------------------------------------------------------


Load Dataset: 100%|██████████| 36/36 [00:28<00:00,  1.25it/s]


Epoch[0500/5000] Loss_D: 0.0516, Loss_G: 8.0418, elapsed: 32.8, remaining: 295.6
Epoch[1000/5000] Loss_D: 2.3743, Loss_G: 5.9121, elapsed: 26.3, remaining: 210.4
Epoch[1500/5000] Loss_D: 0.0145, Loss_G: 26.9942, elapsed: 28.2, remaining: 197.6
Epoch[2000/5000] Loss_D: 0.0238, Loss_G: 10.2667, elapsed: 27.7, remaining: 166.1
Epoch[2500/5000] Loss_D: 0.0017, Loss_G: 18.1709, elapsed: 28.5, remaining: 142.8
Epoch[3000/5000] Loss_D: 0.0106, Loss_G: 5.6145, elapsed: 27.4, remaining: 109.6
Epoch[3500/5000] Loss_D: 0.0142, Loss_G: 7.2781, elapsed: 28.5, remaining: 85.5
Epoch[4000/5000] Loss_D: 0.0217, Loss_G: 6.7062, elapsed: 28.1, remaining: 56.2
Epoch[4500/5000] Loss_D: 0.0475, Loss_G: 16.1553, elapsed: 27.4, remaining: 27.5
Epoch[5000/5000] Loss_D: 0.1012, Loss_G: 8.2037, elapsed: 28.1, remaining: 0.1


100%|██████████| 100/100 [00:01<00:00, 93.10it/s]
